# <center>Curso de Modelagem de Dados para IA - PARTE 7</center>

<img src="img/image.jpg" alt="Drawing" style="width: 300px;"/>


## Preparação dos Dados
Mesmo depois de importar os dados para qualquer idioma ou sistema de análise com o qual esteja trabalhando, geralmente há a necessidade de vários tipos de limpeza e pré-processamento de dados, a fim de preparar os dados para análises posteriores. Existem tipos recorrentes de tratamento de dados que surgem em muitos contextos diferentes, que serão abordados no decorrer deste tutorial:

- Limpeza de dados
- Extração e reorganização de dados
- Aumento de dados, como adição de dados derivados com base nos dados brutos importados ou integração de vários conjuntos de dados relacionados
- Aplicação de operações agregadas em dados e/ou subconjuntos de dados agrupados
- Filtragem de subconjuntos de dados com base em vários critérios

Muitas vezes, os dados adquiridos precisarão ser limpos antes que se possa começar a trabalhar com eles. (Este é o lado menos glamoroso do campo da ciência de dados, mas amplamente reconhecido como uma parte central e muitas vezes complicada do processo.) Serão apresentadas algumas ferramentas úteis dentro do **pandas** para limpar dados que foram importados para um dataframe, bem como algumas estratégias gerais para lidar com problemas em dados.

### A necessidade de limpeza de dados
Usa-se o termo um tanto inespecífico "limpeza" para nos referirmos a uma variedade de operações necessárias para colocar os dados de entrada em um estado útil para análise posterior. Isso pode incluir lidar com:

- Dados ausentes
- Representações de dados fora do padrão
- Dados inconsistentes

### Comentários gerais sobre dados ausentes
Quando falamos de dados "ausentes", geralmente estamos nos referindo a conjuntos de dados que são suficientemente regulares em estrutura (por exemplo, arrays, tabelas, dataframes) e que, em princípio, deveriam ser completamente preenchidos com dados, mas na verdade não o são. (Como tal, estamos fazendo uma distinção de dados "irregulares", que não têm a mesma estrutura regular implícita e que podem precisar ser pré-processados usando interpolação, krigagem, suavização ou ajuste para torná-los úteis para análise posterior. ) Os dados podem estar ausentes de um conjunto de dados por vários motivos, mas diferentes pacotes de software podem acomodar mais ou menos dados ausentes. Arrays *NumPy* e dataframes *Pandas*, por exemplo, têm uma forma definida, então não é possível definir um array ou um dataframe que tenha diferentes números de elementos em cada linha. No entanto, pode-se preencher as entradas ausentes com um valor destinado a indicar que a entrada está ausente; O Pandas, por exemplo, usa o formato de não-número <span style="font-family: 'Courier'">np.nan</span> do NumPy para refletir dados ausentes em uma série ou dataframe. O NumPy, por outro lado, apresenta uma estrutura de dados mais complicada — arrays mascarados — que podem ser usados para dar suporte a arrays com entradas ausentes.

### Para lidar com dados ausentes, existem três grandes tipos de estratégias:

- Ignore os dados ausentes
- Elimine entradas com dados em falta
- Preencha os dados ausentes (ou tente preenchê-los na fonte)

Ignorar dados ausentes é realmente apenas uma estratégia útil se você entender como eles serão ignorados. Em várias operações de agregação (por exemplo, soma, média, desvio padrão, etc.), o Pandas simplesmente pulará os dados ausentes como se não estivessem lá: para calcular a média de uma coluna, por exemplo, o Pandas calcula a média de entradas não faltantes nessa coluna. Da mesma forma, uma operação agregada em uma matriz mascarada NumPy incluirá apenas as entradas que não são mascaradas.

Uma estratégia apropriada para descartar entradas com dados ausentes realmente depende de quais perguntas você deseja fazer. Por exemplo, quando formos trabalhar com os dados de incêndios florestais, iremos calcular a duração de um incêndio calculando o número de dias entre quando o incêndio foi identificado pela primeira vez (<span style="font-family: 'Courier'">ALARM_DATE</span>) e quando foi contido (<span style="font-family: 'Courier'">CONT_DATE</span>). Se uma ou ambas as datas estiverem faltando, não podemos calcular uma duração para esse incêndio, então faz sentido (para fins de obter informações sobre a duração do incêndio) considerar apenas incêndios para os quais ambas as datas são conhecidas e descartar todo o resto a partir dessa análise.

### Limpando os dados do Dataset sobre Incêndios Florestais
Embora o conjunto de dados sobre incêncios florestais seja extremamente útil, veremos abaixo que existem algumas falhas nos dados que precisam ser corrigidas antes que possamos mergulhar nas análises. Às vezes, esses tipos de problemas são óbvios desde o início (por exemplo, talvez um arquivo csv esteja sem um cabeçalho no topo), mas geralmente eles só são descobertos depois que você começa a trabalhar com os dados e percebe algumas peculiaridades. Trabalharemos com alguns dos problemas encontrados no conjunto de dados sobre incêndios florestais abaixo e concluiremos esta parte com algumas ideias mais amplas sobre a limpeza de dados.

Anteriormente, usamos uma função pandas para ler os dados sobre incêndios florestais da planilha chamada "2016", que estava armazenada em uma planilha do Excel. Vamos lê-lo novamente e analisá-lo.

In [1]:
import pandas as pd

df16 = pd.read_excel('Fires_100.xlsx', sheet_name='2016')
df16.head()

,YEAR,STATE,AGENCY,UNIT_ID,FIRE_NAME,INC_NUM,ALARM_DATE,CONT_DATE,CAUSE,COMMENTS,REPORT_AC,GIS_ACRES,C_METHOD,OBJECTIVE
0,1878,California,Contract County,Los Angeles County,,0,<Null>,<Null>,14 - Unknown / Unidentified,NaN,<Null>,59468.8800,<Null>,Suppression (Wildfire)
1,1895,California,Contract County,Los Angeles County,,0,<Null>,<Null>,14 - Unknown / Unidentified,NaN,<Null>,628.1136,<Null>,Suppression (Wildfire)
2,1896,California,Contract County,Los Angeles County,,0,<Null>,<Null>,14 - Unknown / Unidentified,NaN,<Null>,7531.7170,<Null>,Suppression (Wildfire)
3,1898,California,Contract County,Los Angeles County,,0,<Null>,<Null>,14 - Unknown / Unidentified,NaN,<Null>,1660.1030,<Null>,Suppression (Wildfire)
4,1898,California,Contract County,Los Angeles County,,0,<Null>,<Null>,14 - Unknown / Unidentified,NaN,<Null>,428.4387,<Null>,Suppression (Wildfire)


In [2]:
df16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11795 entries, 0 to 11794
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   YEAR        11789 non-null  object 
 1   STATE       11795 non-null  object 
 2   AGENCY      11795 non-null  object 
 3   UNIT_ID     11794 non-null  object 
 4   FIRE_NAME   11769 non-null  object 
 5   INC_NUM     11720 non-null  object 
 6   ALARM_DATE  11795 non-null  object 
 7   CONT_DATE   11795 non-null  object 
 8   CAUSE       11795 non-null  object 
 9   COMMENTS    1961 non-null   object 
 10  REPORT_AC   11795 non-null  object 
 11  GIS_ACRES   11795 non-null  float64
 12  C_METHOD    11795 non-null  object 
 13  OBJECTIVE   11795 non-null  object 
dtypes: float64(1), object(13)
memory usage: 1.3+ MB


Comparar as informações fornecidas por <span style="font-family: 'Courier'">head()</span> e <span style="font-family: 'Courier'">info()</span> é útil pois pode-se perceber algumas informações interessantes sobre os dados. As informações em <span style="font-family: 'Courier'">df.head()</span> sugerem que há algumas entradas ausentes nas colunas <span style="font-family: 'Courier'">ALARM_DATE</span> e <span style="font-family: 'Courier'">CONT_DATE</span> (codificadas como <<span style="font-family: 'Courier'">Null</span>>), mas as informações de resumo sugerem que essas entradas são todas "não nulas". O Pandas pode lidar com dados ausentes, mas acontece que <<span style="font-family: 'Courier'">Null</span>> é uma representação não padrão para dados ausentes. Portanto, o Pandas o interpreta como a cadeia de caracteres não nula "<<span style="font-family: 'Courier'">Null</span>>". Felizmente, a função <span style="font-family: 'Courier'">pd.read_excel()</span> tem uma opção para especificar valores ausentes adicionais. Se você estiver acompanhando no notebook Jupyter, digite <span style="font-family: 'Courier'">pd.read_excel?</span> ou <span style="font-family: 'Courier'">help(pd.read_excel)</span> para inspecionar a documentação desta função. Você verá que existe uma opção para *read_excel* que permite especificar dados adicionais como nulos, ausentes ou na (não disponível) chamados <span style="font-family: 'Courier'">na_values</span>.

In [3]:
df = pd.read_excel('Fires_100.xlsx', sheet_name='2016', na_values=['<Null>'])

Agora, se executarmos novamente <span style="font-family: 'Courier'">df.head()</span> e <span style="font-family: 'Courier'">df.info()</span>, veremos que há muito menos instâncias de dados não nulos. Curiosamente, <span style="font-family: 'Courier'">ALARM_DATE</span> é interpretado como um "objeto" (leia-se: string), e <span style="font-family: 'Courier'">CONT_DATE</span> é interpretado como uma data (<span style="font-family: 'Courier'">datetype64[ns]</span>). A razão para essa discrepância parece ser o fato de que dois dos <span style="font-family: 'Courier'">ALARM_DATEs</span> ocorreram antes de 1900 e o Pandas não os interpretam corretamente como datas, deixando-os como *strings*. Seria bom se ambos fossem interpretados como datas para que pudéssemos operá-los como tal. Felizmente, podemos fazer isso usando a função pd.to_datetime, após lermos o dataframe ou durante o próprio processo de leitura, atribuindo explicitamente um conversor a cada um dos campos de dados:

In [5]:
df = pd.read_excel('Fires_100.xlsx', sheet_name='2016', na_values=['<Null>'], \
    converters={'ALARM_DATE': pd.to_datetime, 'CONT_DATE': pd.to_datetime})

Embora fosse conveniente poder fazer a conversão de data e hora diretamente como parte da entrada do arquivo, essa nem sempre é a abordagem ideal, conforme será discutido na próxima parte deste tutorial. Conforme discutido nessa parte, se os dados do *timestamp* de data/hora estiverem em um formato não padrão, provavelmente seja desejável fornecer uma string de formato específico para orientar a conversão, em vez de depender do Pandas para inferir um timestamp de data/hora, o que pode levar muito mais tempo para um arquivo grande. Se você quiser seguir essa abordagem, poderá primeiro chamar <span style="font-family: 'Courier'">pd.read_excel</span> para ler primeiro os dados do arquivo e, em seguida, fazer a conversão de data e hora para as colunas de dados de interesse chamando <span style="font-family: 'Courier'">pd.to_datetime</span> com a string de formato apropriada.

### Comentários gerais sobre dados inconsistentes
Os dados podem ser "inconsistentes" de várias maneiras, incluindo:

- Ortografias alternativas/erradas ou texto em formato maiúsculo
- Dados em uma tabela que são principalmente de um tipo (por exemplo, <span style="font-family: 'Courier'">floats</span>), mas com algumas instâncias de entradas com texto adicional incluído
- Dados em tabelas diferentes que deveriam ser vinculadas por meio de um conjunto de chaves compartilhadas, mas existem algumas chaves que ocorrem em apenas uma das tabelas.

Infelizmente, não há uma solução abrangente para corrigir esses problemas, muito menos para detectá-los. Claramente, é melhor corrigir esses problemas na fonte de dados. Se você precisar corrigir problemas com dados, faça isso em seu código em vez de em um arquivo de dados que alguém forneceu para você, pois quaisquer alterações feitas no arquivo de dados serão perdidas se você receber uma versão atualizada. No entanto, fornecemos aqui algumas sugestões gerais ou estratégias que podem ser úteis nesse processo.

- Se seus dados estiverem em um dataframe, use <span style="font-family: 'Courier'">df.info()</span> para imprimir o tipo de cada coluna. Se um tipo unificador não puder ser inferido por pandas, os tipos de coluna serão inferidos genericamente como do tipo "objeto" (leia-se "string"). Se você acha que uma determinada coluna deve ser de algum outro tipo (por exemplo, <span style="font-family: 'Courier'">int</span>, <span style="font-family: 'Courier'">float</span>, <span style="font-family: 'Courier'">datetime64</span>), tente identificar quais entradas na coluna podem estar contribuindo para a falta de coerência de tipo.
- Se houverem letras maiúsculas em posições alternadas no texto, converter tudo para um estado uniforme (por exemplo, para letras minúsculas, usando um método <span style="font-family: 'Courier'">str.lower()</span>) pode ser útil como forma fácil de normalização de texto. Para problemas mais complicados na normalização e/ou lematização de texto, vários pacotes NLP (como *NLTK*, *spacy* ou *textblob*) fornecem suporte mais avançado.
- Se seus dados estiverem em um dataframe, pode ser útil examinar o conjunto de entradas únicas em uma determinada coluna, usando o método <span style="font-family: 'Courier'">unique()</span> em um dataframe ou série (ou seja, coluna). Isso é mais útil para dados categóricos ou de string do que para números. Se houver variantes de ortografia que devam ser unificadas, elas podem ser identificadas mais facilmente filtrando para o conjunto exclusivo; para um dataframe df com uma coluna chamada 'alguma_coluna' que você gostaria de examinar, o seguinte comando pode ser útil: <span style="font-family: 'Courier'">sorted(df['alguma_coluna'].unique())</span>. Ao classificar as entradas da coluna, é mais provável que você encontre variantes de ortografia próximas umas das outras.
- Para variações mais complicadas na ortografia ou para ajudar a rastrear possíveis erros de entrada de dados, às vezes pode ser útil usar ferramentas de processamento de string para identificar partes de texto intimamente relacionadas. O pacote ***python-Levenshtein***, por exemplo, fornece uma maneira conveniente e leve de calcular distâncias e semelhanças de edição de strings, o que pode ser útil no diagnóstico de problemas com dados de texto.
- Os objetos do tipo <span style="font-family: 'Courier'">set</span> do Python podem ser úteis para diagnosticar inconsistências nos dados. Se dois dataframes devem ter um conjunto de chaves correspondentes (ou seja, um conjunto compartilhado de entradas em uma coluna especificada), mas na verdade não têm, as operações de conjunto podem facilmente identificar discrepâncias, por exemplo, <span style="font-family: 'Courier'">set(df1['some_column'])-set( df2['some_column'])</span> retornará tudo na coluna do dataframe em df1 que não está na coluna correspondente em df2.
- A plotagem de dados é, obviamente, um método consagrado pelo tempo para usar os recursos de reconhecimento de padrões do cérebro humano para identificar rapidamente discrepâncias ou outras partes estranhas de um conjunto de dados.